# Capstone Project: Segmenting and Clustering Neighbourhoods in Toronto

## 1.- Install Libraries and packages

In [1]:
# Gabriel: Install the beautifulsoup package; which needs to be install to use bs4
! pip install beautifulsoup4

In [2]:
# Gabriel: Install the lxml parser which needs to be install to use bs4
! pip install lxml

In [3]:
# Gabriel: Install the html5lib parser which needs to be install to use bs4
! pip install html5lib

In [4]:
#Installing libraries
import numpy as np
import pandas as pd

## 2.-Scrape the Data from wikipedia

In [5]:
# Scrape the data from the web link
Data=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
Toronto_Data=Data[0]
Toronto_Data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## 3.- Ignore cells with a borough that is Not assigned.

In [6]:
# To Eliminate the lines with Borough not assigned first we make a quick view to our data
Toronto_Data.describe()

,Postal Code,Borough,Neighbourhood
count,180,180,180
unique,180,11,100
top,M1M,Not assigned,Not assigned
freq,1,77,77


In [7]:
na=(Toronto_Data['Borough']=='Not assigned').value_counts() # Here i count how many "Not Assigned" Borough exists
na

False    103
True      77
Name: Borough, dtype: int64

In [8]:
# Eliminate the lines with Borough not assigned
Toronto_Data['Borough'].replace('Not assigned',np.nan,inplace=True) #Replace the "Not assinged by Nan"
Toronto_Data.dropna(subset=["Borough"], axis=0, inplace=True) #Drop Nan
Toronto_Data.describe() # I check that the Not Assigned rows were eliminated from our data. I used to have 180 rows in which 103 rows had an Assigned borough and 77 were not assigned

,Postal Code,Borough,Neighbourhood
count,103,103,103
unique,103,10,99
top,M4G,North York,Downsview
freq,1,24,4


In [9]:
na=(Toronto_Data['Borough']=='Not assigned').value_counts() # As a double check, i count how many rows are not assigned
na # we can see that there are not any 'not assigned' rows.

False    103
Name: Borough, dtype: int64

In [10]:
#Check on my new Toronto Data
Toronto_Data

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
# Reset Index
Toronto_Data.reset_index(drop=True,inplace=True)
Toronto_Data.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 3.- Replace Neighbourhoods are not assigned with their Borough

In [12]:
# Double check to see if there is any "Not Assigend" in the Neighbourhood column
neigh_na=Toronto_Data[['Neighbourhood']]=='Not assigned'
neigh_na.value_counts()

Neighbourhood
False            103
dtype: int64

#### We see that there is not "Not assigned" neighbourhoods to replace

## 4.- Group neighbourhoods by Postal Codes

In [13]:
Toronto_Data = Toronto_Data.groupby('Postal Code',as_index=False).agg(lambda x: ','.join(set(x.dropna()))) # Group by function to group by postal code
Toronto_Data # in the previous line of code we ad the "agg" object to separate the sets by a comma

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


## 5.- Check the size of the data

In [14]:
Toronto_Data.shape

(103, 3)

# PART 2 OF THE PROJECT

In [15]:
#Import the Geographical coordinates from the csv files
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [16]:
# Merge the tables
Toronto_Data=pd.merge(left=Toronto_Data, right=geo_data, on='Postal Code')
Toronto_Data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# PART 3 OF THE PROJECT

## 1.- Explore the Toronto neighbourhood data

In [17]:
Toronto_Data[['Borough']].value_counts().to_frame()

,0
Borough,
North York,24
Downtown Toronto,19
Scarborough,17
Etobicoke,12
Central Toronto,9
West Toronto,6
York,5
East York,5
East Toronto,5


## 2.- Identify the Borough that contains the word "Toronto"

In [18]:
TorontoClusterData = Toronto_Data[Toronto_Data['Borough'].str.contains('Toronto')].reset_index(drop=True)
TorontoClusterData.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [19]:
## I just wanted to check a simple statistic of the data we have
print(TorontoClusterData.shape)
TorontoClusterData.describe(include='all')

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
count,39,39,39,39.000000,39.000000
unique,39,4,39,NaN,NaN
top,M5A,Downtown Toronto,"Business reply mail Processing Centre, South C...",NaN,NaN
freq,1,19,1,NaN,NaN
mean,NaN,NaN,NaN,43.667135,-79.389873
std,NaN,NaN,NaN,0.023478,0.037451
min,NaN,NaN,NaN,43.628947,-79.484450
25%,NaN,NaN,NaN,43.649765,-79.405678
50%,NaN,NaN,NaN,43.662301,-79.387383
75%,NaN,NaN,NaN,43.677957,-79.376474


## 3.- Map of Toronto Neighbourhood

In [20]:

#!conda install -c conda-forge geopy --yes
#!pip install folium
from geopy.geocoders import Nominatim
import folium

# Extract geograpical coordinate of Toronto
Address = 'Toronto'
Geo = Nominatim(user_agent="ny_explorer")
Location = Geo.geocode(Address)
Latitude = Location.latitude
Longitude = Location.longitude

# Create map of Toronoto and Boroughs' markers
Toronto_Map = folium.Map(location=[Latitude, Longitude], zoom_start=12)
for x, y, b, n in zip(TorontoClusterData['Latitude'], TorontoClusterData['Longitude'],\
                      TorontoClusterData['Borough'], TorontoClusterData['Neighbourhood']):
    Label = '{}, {}'.format(n, b)
    Label = folium.Popup(Label, parse_html=True)
    folium.CircleMarker(
        [x, y],
        radius=5,
        popup=Label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_Map);
Toronto_Map

### Foursquare Api Credentials

In [21]:
CLIENT_ID = 'JV5CJLLWGQM0UL3UZMWWOVLPMMKQRVBQ3RH4KWOGO1IJNUVW' # your Foursquare ID
CLIENT_SECRET = 'RH5UB3QUBJV2A2RHVRK1QVIL4FOCFBWQSUDE3ZKAJSHJ1DJ5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JV5CJLLWGQM0UL3UZMWWOVLPMMKQRVBQ3RH4KWOGO1IJNUVW
CLIENT_SECRET:RH5UB3QUBJV2A2RHVRK1QVIL4FOCFBWQSUDE3ZKAJSHJ1DJ5


### Take a look at one neighbourhood

In [22]:
TorontoClusterData.loc[0, 'Neighbourhood']

'The Beaches'

In [23]:
neighborhood_latitude = TorontoClusterData.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = TorontoClusterData.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = TorontoClusterData.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Make a function to get the venues and categories of the Neighbourhoods

In [24]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
# type your answer here
Toronto_venues = getNearbyVenues(names=TorontoClusterData['Neighbourhood'],
                                   latitudes=TorontoClusterData['Latitude'],
                                   longitudes=TorontoClusterData['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [26]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1605, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [27]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [28]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.016667,0.000000,0.016667
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Let's print each neighborhood along with the top 10 most common venues

In [30]:
num_top_venues = 10

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1         Cocktail Bar  0.05
2       Farmers Market  0.04
3   Seafood Restaurant  0.04
4          Cheese Shop  0.04
5           Restaurant  0.04
6             Beer Bar  0.04
7               Bakery  0.04
8  Sporting Goods Shop  0.02
9               Bistro  0.02


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.14
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3            Intersection  0.05
4                 Stadium  0.05
5  Furniture / Home Store  0.05
6            Climbing Gym  0.05
7                     Bar  0.05
8              Restaurant  0.05
9               Nightclub  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2               Brewery  0.06
3               Butcher  0.06
4        

First, let's write a function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues=10):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Restaurant,Farmers Market,Beer Bar,Bakery,Seafood Restaurant,Bistro,Basketball Stadium
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio,Portuguese Restaurant,Indian Restaurant


#### Start Clusterization

In [33]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 2, 0, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = TorontoClusterData

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Health Food Store,Asian Restaurant,Trail,Neighborhood,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Cosmetics Shop,Juice Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Fast Food Restaurant,Pizza Place,Park,Brewery,Sandwich Place,Liquor Store,Fish & Chips Shop,Italian Restaurant,Restaurant,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Brewery,Café,Gastropub,American Restaurant,Bakery,Yoga Studio,Neighborhood,Cheese Shop,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Swim School,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


## 4.- Create a map of the clusters

In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

### Cluster 1

In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,0,Garden,Health & Beauty Service,Home Service,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio


### Cluster 2

In [37]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Pub,Health Food Store,Asian Restaurant,Trail,Neighborhood,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
1,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Cosmetics Shop,Juice Bar
2,East Toronto,1,Fast Food Restaurant,Pizza Place,Park,Brewery,Sandwich Place,Liquor Store,Fish & Chips Shop,Italian Restaurant,Restaurant,Steakhouse
3,East Toronto,1,Coffee Shop,Brewery,Café,Gastropub,American Restaurant,Bakery,Yoga Studio,Neighborhood,Cheese Shop,Clothing Store
5,Central Toronto,1,Gym / Fitness Center,Sandwich Place,Park,Pizza Place,Breakfast Spot,Department Store,Hotel,Food & Drink Shop,Doner Restaurant,Donut Shop
6,Central Toronto,1,Coffee Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Shoe Store,Fast Food Restaurant,Spa,Sporting Goods Shop,Salon / Barbershop,Mexican Restaurant
7,Central Toronto,1,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Gym,Café,Thai Restaurant,Italian Restaurant,Park
9,Central Toronto,1,Coffee Shop,Fried Chicken Joint,Pizza Place,Liquor Store,Restaurant,Pub,Bank,Bagel Shop,Supermarket,Sushi Restaurant
11,Downtown Toronto,1,Coffee Shop,Restaurant,Pub,Bakery,Pet Store,Park,Pizza Place,Italian Restaurant,Café,Beer Store
12,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Fast Food Restaurant,Restaurant,Pub,Café,Yoga Studio,Hotel


### Cluster 3

In [38]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Park,Bus Line,Swim School,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
8,Central Toronto,2,Park,Lawyer,Trail,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
10,Downtown Toronto,2,Park,Playground,Trail,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
